# EDA+資料處理(到資料合併的步驟)

## 匯入所需的函式庫並指定之後跑神經網路所使用的GPU

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector, GRU
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import preprocessing
from sklearn.cluster import KMeans
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## 系統相關設定

In [ ]:
# 設定dataframe在輸出時可以看到所有的欄位
pd.set_option('display.max_columns', None)

## 從資料庫中取得原始資料集(包含所有的欄位)

In [ ]:
import pymysql.cursors
connection = pymysql.connect(host='140.119.9.88',
                             user='isvmsdata',
                             password='dataisvms2018',
                             port = 3306,
                             db='test_cvm',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
df_mysql = pd.read_sql("SELECT * FROM avm_table", con=connection)
connection.close()

In [ ]:
data = df_mysql # 保留df_mysql作為原始資料

## 遺漏值處理 - 把沒有產品名稱的資料去除

In [ ]:
#print(df_mysql.isnull().sum())
data = data.dropna(subset=['產品名稱'])
print(data.isnull().sum())
print("總共清除的資料筆數：",len(df_mysql)-len(data))
print("剩下的資料筆數：", len(data))

In [ ]:
#df_mysql[df_mysql['單位'].isnull()]

## 探索性分析 - 第一部分 (資料合併之前)

In [ ]:
eda_data = data

### (1) 各產品實銷貨量分布(不計實銷貨量為負的產品) (面積圖)

In [ ]:
eda_data_groupbyProduct = eda_data.groupby(['產品名稱']).sum()["實銷貨量"]
eda_data_groupbyProduct = eda_data_groupbyProduct.sort_values(ascending=False).reset_index()['實銷貨量']
eda_data_groupbyProduct = eda_data_groupbyProduct[eda_data_groupbyProduct >= 0]
#eda_data_groupbyProduct.head()

In [ ]:
fig, ax = plt.subplots(figsize = (15,7))
plt.ticklabel_format(style='plain')
eda_data_groupbyProduct.plot.area()
plt.rcParams['font.sans-serif'] = ['simhei']
plt.tick_params(
    axis = 'x',          # changes apply to the x-axis
    which = 'both',      # both major and minor ticks are affected
    bottom = False,      # ticks along the bottom edge are off
    top = False,         # ticks along the top edge are off
    labelbottom = False) 
plt.yticks(fontsize = 16)
plt.ylabel('實銷貨量', fontsize = 20)
ax.set_title("各產品實銷貨量分布(不計實銷貨量為負的產品)(面積圖)", fontsize = 24)

### (2) 各月份整體產品實銷貨量(折線圖)

In [ ]:
# 新增「年/月」的欄位
eda_data['年/月'] = eda_data.apply(lambda x: str(x['年']) + "/" + "%02d"%(x['月']), axis=1)

In [ ]:
eda_data_groupbyDate = eda_data.groupby(['年/月'])["實銷貨量"].sum()
#eda_data_groupbyDate.head()

In [ ]:
fig, ax = plt.subplots(figsize=(18,7))
plt.plot(eda_data_groupbyDate.index, eda_data_groupbyDate)
plt.rcParams['font.sans-serif'] = ['simhei']
#plt.legend(loc='upper right', fontsize=16)
plt.xlabel('年/月', fontsize=16)
plt.xticks(rotation=60, fontsize=14)
plt.ylabel('實銷貨量', fontsize=16)
plt.yticks(fontsize=14)
plt.axvline(x=12, color='r', linestyle='--')
plt.axvline(x=24, color='r', linestyle='--')
plt.axvline(x=36, color='r', linestyle='--')
ax.set_title("各月份整體產品實銷貨量(折線圖)", fontsize=24)
plt.show()

### (3) 各部門整體產品實銷貨比例(圓餅圖)

In [ ]:
eda_data_groupbyDepartment = eda_data.groupby(["部門名稱"])["實銷貨量"].sum()

In [ ]:
# 把比例過低的部門(廠務部、行銷部、生管課)合併成「其他部門」
eda_data_groupbyDepartment2 = eda_data_groupbyDepartment.sort_values(ascending=False)[:9].copy()
eda_data_groupbyDepartment2.loc['其他部門(廠務部+行銷部+生管課)'] = (
    eda_data_groupbyDepartment.sort_values(ascending=False)[9:].sum()
)
#eda_data_groupbyDepartment2

In [ ]:
plt.figure(figsize=(8,11))    # 顯示圖框架大小

labels =  eda_data_groupbyDepartment2.index                    # 製作圓餅圖的類別標籤
separated = (0, 0, 0, 0, 0, 0, 0, 0.1, 0.2, 0.3)                  # 依據類別數量，分別設定要突出的區塊
size = eda_data_groupbyDepartment2      # 製作圓餅圖的數值來源

plt.pie(
    size,                           # 數值
    labels = labels,                # 標籤
    autopct = "%.1f%%",            # 將數值百分比並留到小數點一位           # explode 設定分隔的區塊位置
    pctdistance = 0.6,              # 數字距圓心的距離
    textprops = {"fontsize" : 20},  # 文字大小
    startangle=-270,
    explode=separated
)
 
plt.axis('equal')                                          # 使圓餅圖比例相等
plt.title("各部門整體產品實銷貨比例(圓餅圖)", {"fontsize" : 24})  # 設定標題及其文字大小
#plt.legend(loc = "right", bbox_to_anchor=(1.6, 0.5), fontsize = 20)            # 設定圖例及其位置為最佳

### (4) 實銷貨量前五大客戶占比(圓餅圖)

In [ ]:
eda_data_groupbyCustomer = eda_data.groupby(['客戶簡稱']).sum()["實銷貨量"]

In [ ]:
# 把第六名後的客戶合併成「其他客戶」
eda_data_groupbyCustomer2 = eda_data_groupbyCustomer.sort_values(ascending=False)[:5].copy()
eda_data_groupbyCustomer2.loc['其他客戶'] = (
    eda_data_groupbyCustomer.sort_values(ascending=False)[5:].sum()
)
#eda_data_groupbyCustomer2

In [ ]:
plt.figure(figsize=(8,8))    # 顯示圖框架大小

labels =  eda_data_groupbyCustomer2.index                    # 製作圓餅圖的類別標籤
separated = (0, 0, 0, 0.1, 0.2, 0)                  # 依據類別數量，分別設定要突出的區塊
size = eda_data_groupbyCustomer2        # 製作圓餅圖的數值來源

plt.pie(size,                           # 數值
        labels = labels,                # 標籤
        autopct = "%.1f%%",            # 將數值百分比並留到小數點一位           # explode 設定分隔的區塊位置
        pctdistance = 0.6,              # 數字距圓心的距離
        textprops = {"fontsize" : 20},  # 文字大小
        startangle=-270,
        explode=separated
       )
 
plt.axis('equal')                                          # 使圓餅圖比例相等
plt.title("實銷貨量前五大客戶占比(圓餅圖)", {"fontsize" : 24})  # 設定標題及其文字大小
#plt.legend(loc = "right", bbox_to_anchor=(1.6, 0.5), fontsize = 20)            # 設定圖例及其位置為最佳

### --- 以下為最後沒有採用的統計圖 --- 

### 各產品實銷貨量-TOP10(長條圖)

In [ ]:
eda_data_groupbyProduct = eda_data.groupby(['產品名稱']).sum()["實銷貨量"]
eda_data_groupbyProduct.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.ticklabel_format(style='plain')
eda_data_groupbyProduct.sort_values(ascending=False).iloc[:10].plot.bar()
plt.rcParams['font.sans-serif'] = ['simhei']
plt.xticks(rotation=45, fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('產品名稱', fontsize=20)
plt.ylabel('實銷貨量', fontsize=20)
ax.set_title("各產品實銷貨量-TOP10(長條圖)", fontsize=24)
for i, v in enumerate(eda_data_groupbyProduct.sort_values(ascending=False).iloc[:10].values):
    ax.text(i, v, int(v), horizontalalignment='center', fontsize = 16)

### 各業務員實銷貨量-TOP10(長條圖)

In [ ]:
eda_data_groupbySalesman = eda_data.groupby(['業務員名稱']).sum()["實銷貨量"]
eda_data_groupbySalesman.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.ticklabel_format(style='plain')
eda_data_groupbySalesman.sort_values(ascending=False).iloc[:10].plot.bar() # 只看前十名
plt.rcParams['font.sans-serif'] = ['simhei']
plt.xticks(rotation=0, fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('業務員名稱', fontsize=20)
plt.ylabel('實銷貨量', fontsize=20)
ax.set_title("各業務員實銷貨量-TOP10(長條圖)", fontsize=24)
for i, v in enumerate(eda_data_groupbySalesman.sort_values(ascending=False).iloc[:10].values):
    ax.text(i, v, int(v), horizontalalignment='center', fontsize = 16)

### 各單位實銷貨量(長條圖)

In [ ]:
eda_data_groupbyUnit = eda_data.groupby(['單位']).sum()["實銷貨量"]
eda_data_groupbyUnit.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.ticklabel_format(style='plain')
eda_data_groupbyUnit.sort_values(ascending=False).plot.bar()
plt.rcParams['font.sans-serif'] = ['simhei']
plt.xticks(rotation=45, fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('單位名稱', fontsize=20)
plt.ylabel('實銷貨量', fontsize=20)
ax.set_title("各單位實銷貨量(長條圖)", fontsize=24)

## 特徵工程 - 原始變數

### (1) 把「年」、「月」欄位合併成「年/月」

In [ ]:
data['年/月'] = data.apply(lambda x: str(x['年']) + "/" + "%02d"%(x['月']), axis=1)
data = data.drop(['年', '月'], axis = 1)

### (2) 把非二元的類別變數轉成獨熱編碼 (目前選用的非二元的類別變數：月份、銷售單價級距1)

In [ ]:
# 新增「月份」獨熱編碼的欄位
data['月'] = data.apply(lambda x: str(x['年/月'])[-2:], axis=1)
month = pd.get_dummies(data["月"])
data = data.join(month)
data = data.drop(["月"], axis = 1)

In [ ]:
# 新增「銷售單價級距1」獨熱編碼的欄位
price_interval = pd.get_dummies(data["銷售單價級距1"])
data = data.join(price_interval)

In [ ]:
data.head()

### (3) 將原始資料集按照「產品名稱、年、月」合併，並取出之後會用到的欄位

In [ ]:
data_sum = data.groupby(["產品名稱", "年/月"]).sum()
data_avg = data.groupby(["產品名稱", "年/月"]).mean()

In [ ]:
data_continuous = data_sum[["實銷貨量", "含稅總成本", "成本合計", "銷貨淨額", "淨利", "贈送量", "促銷量"]]

In [ ]:
data_discrete = data_avg[data_avg.columns[36:]] # 01(一月)後面的所有類別變數

In [ ]:
data_contin_discrete = data_continuous.join(data_discrete)
data_contin_discrete.head()

### (4) 在合併後的資料集中新增「各部門(當期該產品)實銷貨量比例」的欄位

In [ ]:
salesVolumn_groupbyDateProductDepartment = pd.DataFrame(data.groupby(["產品名稱", "年/月", "部門名稱"])["實銷貨量"].sum())

In [ ]:
salesVolumn_groupbyDateProduct = pd.DataFrame(data.groupby(["產品名稱", "年/月"])["實銷貨量"].sum())

In [ ]:
salesVolumn_merge = pd.merge(
    salesVolumn_groupbyDateProductDepartment, 
    salesVolumn_groupbyDateProduct, 
    on = ['產品名稱', "年/月"], 
    how = 'left'
)
salesVolumn_merge['部門名稱'] = salesVolumn_groupbyDateProductDepartment.reset_index()['部門名稱'].values

In [ ]:
salesVolumn_merge

In [ ]:
department = pd.get_dummies(salesVolumn_merge["部門名稱"])
#department.head()

In [ ]:
department["實銷貨量_x"] = salesVolumn_merge["實銷貨量_x"]
department["實銷貨量_y"] = salesVolumn_merge["實銷貨量_y"]
department["部門名稱"] = salesVolumn_merge["部門名稱"]
department["部門別_實銷貨量"] = department["實銷貨量_x"] / department["實銷貨量_y"]

In [ ]:
department.head()

In [ ]:
department = department.fillna(0)

In [ ]:
department_index_list = department.columns[:12]
for i in department_index_list:
    department[i] = department[i] * department["部門別_實銷貨量"]
department = department.reset_index()

In [ ]:
department_data = department.groupby(["產品名稱","年/月"]).sum().drop(["實銷貨量_x","實銷貨量_y","部門別_實銷貨量"], axis = 1)

In [ ]:
# 查看有無inf或是-inf的狀況發生
department_data[np.isinf(department_data).any(1)]

In [ ]:
# 將出現inf或-inf的資料用0去代替
department_data = department_data.replace([np.inf, -np.inf], 0)

In [ ]:
# 完成各部門的實銷貨量比例
department_data

In [ ]:
# 產品尚未在沒有該日期的資料補零的版本
data_processd_originalVariableOnly = pd.merge(
    data_contin_discrete, 
    department_data, 
    on = ["產品名稱", "年/月"], 
    how = "left")

In [ ]:
data_processd_originalVariableOnly

### (5) 把合併後的資料集的日期補齊(每個產品要有2014/01至2017/12，共48期的資料)，如果產品在該日期沒有資料，則將所有欄位全部補0

In [ ]:
# 建立一個包含所有「產品名稱、年、月」組合的key dataframe
df_list = []
product_list = data['產品名稱'].unique()
date_list = data['年/月'].unique()
for product in product_list:
    df_i = {'年/月': date_list, '產品名稱':[product]*48}
    df_i = pd.DataFrame(df_i)
    df_list.append(df_i)
key_productAndDate = pd.concat(df_list)

In [ ]:
data_processd_originalVariableOnly_fullDate = pd.merge(
    key_productAndDate, data_processd_originalVariableOnly, on = ["產品名稱", "年/月"], how = "left"
)
data_processd_originalVariableOnly_fullDate = data_processd_originalVariableOnly_fullDate.fillna(0)

In [ ]:
data_processd_originalVariableOnly_fullDate

## 特徵工程 - 外部變數

### (1) 取得已處裡完成的外部變數資料，並合併在資料集中

In [ ]:
data_processd_fullDate = data_processd_originalVariableOnly_fullDate

In [ ]:
# 取得已處理完成的外部變數資料
price_index = [98.25, 97.87, 98.05, 98.71, 98.86, 99.37, 99.34, 99.77, 99.72, 99.79, 98.88, 98.54, 97.33, 97.67, 97.45, 97.9, 98.14, 98.81, 98.71, 99.33, 100.01, 100.1, 99.41, 98.67, 98.12, 100.02, 99.41, 99.73, 99.35, 99.7, 99.93, 99.89, 100.35, 101.8, 101.37, 100.34, 100.32, 99.96, 99.59, 99.83, 99.94, 100.7, 100.7, 100.85, 100.84, 101.46, 101.71, 101.56]
retail_revenue = [955, 754, 852, 860, 874, 874, 959, 925, 889, 876, 865, 980, 957, 843, 844, 817, 846, 877, 925, 941, 899, 868, 869, 962, 953, 819, 854, 859, 892, 937, 979, 1033, 919, 921, 923, 1004, 1012, 766, 914, 860, 990, 991, 939, 1012, 985, 901, 955, 997]

price_index_lag1 = [97.95, 98.25, 97.87, 98.05, 98.71, 98.86, 99.37, 99.34, 99.77, 99.72, 99.79, 98.88, 98.54, 97.33, 97.67, 97.45, 97.9, 98.14, 98.81, 98.71, 99.33, 100.01, 100.1, 99.41, 98.67, 98.12, 100.02, 99.41, 99.73, 99.35, 99.7, 99.93, 99.89, 100.35, 101.8, 101.37, 100.34, 100.32, 99.96, 99.59, 99.83, 99.94, 100.7, 100.7, 100.85, 100.84, 101.46, 101.71]
retail_revenue_lag1 = [920, 955, 754, 852, 860, 874, 874, 959, 925, 889, 876, 865, 980, 957, 843, 844, 817, 846, 877, 925, 941, 899, 868, 869, 962, 953, 819, 854, 859, 892, 937, 979, 1033, 919, 921, 923, 1004, 1012, 766, 914, 860, 990, 991, 939, 1012, 985, 901, 955]

is_this_or_next_month_Chinese_New_Year = [1,1,0,0,0,0,0,0,0,0,0,0, 1,1,0,0,0,0,0,0,0,0,0,0, 1,1,0,0,0,0,0,0,0,0,0,1, 1,0,0,0,0,0,0,0,0,0,0,0]
is_this_or_next_month_Lantern_Festival = [1,1,0,0,0,0,0,0,0,0,0,0, 0,1,1,0,0,0,0,0,0,0,0,0, 1,1,0,0,0,0,0,0,0,0,0,0, 1,1,0,0,0,0,0,0,0,0,0,0]
is_this_or_next_month_Tomb_Sweeping_Day = [0,0,1,1,0,0,0,0,0,0,0,0, 0,0,1,1,0,0,0,0,0,0,0,0, 0,0,1,1,0,0,0,0,0,0,0,0, 0,0,1,1,0,0,0,0,0,0,0,0]
is_this_or_next_month_Dragon_Boat_Festival = [0,0,0,0,1,1,0,0,0,0,0,0, 0,0,0,0,1,1,0,0,0,0,0,0, 0,0,0,0,1,1,0,0,0,0,0,0, 0,0,0,1,1,0,0,0,0,0,0,0]
is_this_or_next_month_Hungry_Ghost_Festival = [0,0,0,0,0,0,1,1,0,0,0,0, 0,0,0,0,0,0,1,1,0,0,0,0, 0,0,0,0,0,0,1,1,0,0,0,0, 0,0,0,0,0,0,0,1,1,0,0,0]
is_this_or_next_month_Moon_Festival = [0,0,0,0,0,0,0,1,1,0,0,0, 0,0,0,0,0,0,0,1,1,0,0,0, 0,0,0,0,0,0,0,1,1,0,0,0, 0,0,0,0,0,0,0,0,1,1,0,0]

In [ ]:
#price_index_full = {'物價指數':price_index * 7257}
#retail_revenue_full = {'食品、飲料及菸草製品零售業之營業額':retail_revenue * 7257}
price_index_lag1_full = {'物價指數_延遲一期':price_index_lag1 * 7257}
retail_revenue_lag1_full = {'食品、飲料及菸草製品零售業之營業額_延遲一期':retail_revenue_lag1 * 7257}
is_this_or_next_month_Chinese_New_Year_full = {'這個月或下個月是否是春節':is_this_or_next_month_Chinese_New_Year * 7257}
is_this_or_next_month_Lantern_Festival_full = {'這個月或下個月是否是元宵節':is_this_or_next_month_Lantern_Festival * 7257}
is_this_or_next_month_Tomb_Sweeping_Day_full = {'這個月或下個月是否是清明節':is_this_or_next_month_Tomb_Sweeping_Day * 7257}
is_this_or_next_month_Dragon_Boat_Festival_full = {'這個月或下個月是否是端午節':is_this_or_next_month_Dragon_Boat_Festival * 7257}
is_this_or_next_month_Hungry_Ghost_Festival_full = {'這個月或下個月是否是中元節':is_this_or_next_month_Hungry_Ghost_Festival * 7257}
is_this_or_next_month_Moon_Festival_full = {'這個月或下個月是否是中秋節':is_this_or_next_month_Moon_Festival * 7257}

In [ ]:
#data_processd_fullDate['物價指數'] = pd.DataFrame(price_index_full)
#data_processd_fullDate['食品、飲料及菸草製品零售業之營業額'] = pd.DataFrame(retail_revenue_full)
data_processd_fullDate['物價指數_延遲一期'] = pd.DataFrame(price_index_lag1_full)
data_processd_fullDate['食品、飲料及菸草製品零售業之營業額_延遲一期'] = pd.DataFrame(retail_revenue_lag1_full)
data_processd_fullDate['這個月或下個月是否是春節'] = pd.DataFrame(is_this_or_next_month_Chinese_New_Year_full)
data_processd_fullDate['這個月或下個月是否是元宵節'] = pd.DataFrame(is_this_or_next_month_Lantern_Festival_full)
data_processd_fullDate['這個月或下個月是否是清明節'] = pd.DataFrame(is_this_or_next_month_Tomb_Sweeping_Day_full)
data_processd_fullDate['這個月或下個月是否是端午節'] = pd.DataFrame(is_this_or_next_month_Dragon_Boat_Festival_full)
data_processd_fullDate['這個月或下個月是否是中元節'] = pd.DataFrame(is_this_or_next_month_Hungry_Ghost_Festival_full)
data_processd_fullDate['這個月或下個月是否是中秋節'] = pd.DataFrame(is_this_or_next_month_Moon_Festival_full)

### (2) 新增「日正月營業額」的欄位 (和「食品、飲料及菸草製品零售業之營業額」搭配使用)

In [ ]:
revenue_byMonth = data.groupby(['年/月']).sum()["實銷貨額_含稅"].values.tolist()
#revenue_byMonth

In [ ]:
revenue_byMonth_full = {'日正月營業額':revenue_byMonth * 7257}
data_processd_fullDate['日正月營業額'] = pd.DataFrame(revenue_byMonth_full)

In [ ]:
# 完成特徵工程的部分
data_processd_fullDate

## 儲存處理好的資料集，方便以後存取

In [ ]:
eda_data = data_processd_fullDate

In [ ]:
eda_data.to_csv('eda_data_0728_ver1.csv')

## 清除部分佔記憶體空間大而且之後不會用到的變數，避免server資源消耗太多

In [ ]:
print('start')
import sys
for var, obj in locals().items():
    if sys.getsizeof(obj) > 10000000:
        print(var, sys.getsizeof(obj))

In [ ]:
#del data ,month ,price_interval ,data_sum, data_avg, data_discrete,data_contin_discrete,salesVolumn_merge,department ,data_processd_originalVariableOnly,key_productAndDate,data_processd_originalVariableOnly_fullDate
del _, __, _26, _36, data_processd_fullDate

## 探索性分析 - 第二部分 (資料合併之後)

In [ ]:
list(eda_data)

### (1) 確認補0的資料筆數佔所有資料數量的比例

In [ ]:
"""
# 補0定義是left outer join 後數值全部補0(不是原本就有殘缺的NA補0，因為fillna後就無法分辨原始值本來就是0或因為NA才補0)
# 目前每列 >= 52個0應該就是補0的結果(其他並不會參與補0的11個欄位，其中必定會有值的有：「產品名稱」、「年/月」、「物價指數」、
# 「食品、飲料及菸草製品零售業之營業額」、「日正月營業額」；可能為0或1的有：「這個月或下個月是否是某某節」(共6個節日)
# 所以63個欄位中0的數目>=52欄就是補0的資料)
# print('rows')
zero_count = eda_data[eda_data == 0].count(axis=1)
zero_count_52 = zero_count[zero_count >= 52]
print("補0的資料筆數佔所有資料數量的比例：")
print(len(zero_count_52) / 348336)
print("補0的資料筆數：", len(zero_count_52))
print("348336-補0的資料筆數：{}筆。也就是left outer join前右邊DF(data_inner)的長度".format(348336-len(zero_count_52)))
# 實際沒有在left outer join後才補0的資料筆數應為50274，但那些資料可能就會有補0前就全為0的狀況，
# 因為之前的步驟也有補0和把inf用0替代的狀況
"""

In [ ]:
import matplotlib.pyplot as plt

size = [298062, 50274]
labels = ['有補0', '不用補0']

import matplotlib.pyplot as plt

plt.figure(figsize=(8,8))    # 顯示圖框架大小
                 # 製作圓餅圖的類別標籤
#separated = (0, 0, 0, 0.1, 0.2, 0)                  # 依據類別數量，分別設定要突出的區塊
plt.pie(size,                           # 數值
        #labels = labels,                # 標籤
        autopct = "%.1f%%",            # 將數值百分比並留到小數點一位           # explode 設定分隔的區塊位置
        pctdistance = 0.6,              # 數字距圓心的距離
        textprops = {"fontsize" : 28},  # 文字大小
        startangle=-270
        #explode=separated
       )
 
plt.axis('equal')                                          # 使圓餅圖比例相等
plt.title("依「產品名稱」、「年/月」合併後的資料，\n有補0和不用補0的資料比例", {"fontsize" : 24})  # 設定標題及其文字大小
plt.legend(labels, loc = "right", bbox_to_anchor=(1.35, 0.5), fontsize = 20)            # 設定圖例及其位置為最佳

### (2) 整體產品各變數延遲性(X)與實銷貨量(Y)相關係數實驗(目的在於找出X和Y在t-n期是比較有相關的)

In [ ]:
# 由於有7257項產品，想知道整體產品的延遲性，因此會對7257項產品的相關係數取平均
for i in range(1,5):
    print("延遲{}期".format(i))
    lag_period = i
    gift = np.zeros(7257)          # 各產品贈送量 相關係數array
    promotion = np.zeros(7257)     # 各產品促銷量 相關係數array
    cost_withtax = np.zeros(7257)  # 各產品含稅總成本 相關係數array
    profit = np.zeros(7257)        # 各產品淨利 相關係數array
    labor_cost= np.zeros(7257)     # 各產品成本合計 相關係數array
    sale_revenue = np.zeros(7257)  # 各產品銷貨淨額 相關係數array
    sale_volume = np.zeros(7257)   # 各產品實銷貨量(本身) 相關係數array
    price_index = np.zeros(7257)   # 各產品物價指數 相關係數array
    retail = np.zeros(7257)        # 各產品食品、飲料及菸草製品零售業之營業額 相關係數array
    sales_revenue = np.zeros(7257) # 各產品日正營業額 相關係數array
    for idx, p_name in enumerate(eda_data["產品名稱"].unique()):
        a_product = eda_data[eda_data["產品名稱"] == p_name]
        gift_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["贈送量"][:-lag_period])[0][1]
        if np.isnan(gift_corr):
            gift_corr = 0
        gift[idx] = gift_corr

        promotion_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["促銷量"][:-lag_period])[0][1]
        if np.isnan(promotion_corr):
            promotion_corr = 0
        promotion[idx] = promotion_corr
        
        cost_withtax_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["含稅總成本"][:-lag_period])[0][1]
        if np.isnan(cost_withtax_corr):
            cost_withtax_corr = 0
        cost_withtax[idx] = cost_withtax_corr
        
        profit_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["淨利"][:-lag_period])[0][1]
        if np.isnan(profit_corr):
            profit_corr = 0
        profit[idx] = profit_corr
        
        labor_cost_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["成本合計"][:-lag_period])[0][1]
        if np.isnan(labor_cost_corr):
            labor_cost_corr = 0
        labor_cost[idx] = labor_cost_corr
        
        sale_revenue_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["銷貨淨額"][:-lag_period])[0][1]
        if np.isnan(sale_revenue_corr):
            sale_revenue_corr = 0
        sale_revenue[idx] = sale_revenue_corr
        
        sale_volume_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["實銷貨量"][:-lag_period])[0][1]
        if np.isnan(sale_volume_corr):
            sale_volume_corr = 0
        sale_volume[idx] = sale_volume_corr
        
        price_index_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["物價指數"][:-lag_period])[0][1]
        if np.isnan(price_index_corr):
            price_index_corr = 0
        price_index[idx] = price_index_corr
        
        retail_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["食品、飲料及菸草製品零售業之營業額"][:-lag_period])[0][1]
        if np.isnan(retail_corr):
            retail_corr = 0
        retail[idx] = retail_corr
        
        sales_revenue_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["日正月營業額"][:-lag_period])[0][1]
        if np.isnan(sales_revenue_corr):
            sales_revenue_corr = 0
        sales_revenue[idx] = sales_revenue_corr
        
    print("贈送量 vs 實銷貨量 相關係數")
    print(np.mean(gift))
    print()
    print("促銷量 vs 實銷貨量 相關係數")
    print(np.mean(promotion))
    print()
    print("含稅總成本 vs 實銷貨量 相關係數")
    print(np.mean(cost_withtax))
    print()
    print("淨利 vs 實銷貨量 相關係數")
    print(np.mean(profit))
    print()
    print("成本合計 vs 實銷貨量 相關係數")
    print(np.mean(labor_cost))
    print()
    print("銷貨淨額 vs 實銷貨量 相關係數")
    print(np.mean(sale_revenue))
    print()
    print("實銷貨量(本身) vs 實銷貨量 相關係數")
    print(np.mean(sale_volume))
    print()
    print("物價指數 vs 實銷貨量 相關係數")
    print(np.mean(price_index))
    print()
    print("食品、飲料及菸草製品零售業之營業額 vs 實銷貨量 相關係數")
    print(np.mean(retail))
    print()
    print("日正月營業額 vs 實銷貨量 相關係數")
    print(np.mean(sales_revenue))
    print("-------------------------------------------------------------------------------")

In [ ]:
# 各部門的實銷貨量比例 延遲實驗
# 由於有7257項產品，想知道整體產品的延遲性，因此會對7257項產品的相關係數取平均
# 北營所	台中所	宜蘭所	廠務部	營業一部	營業三部	營業五部	營業六部	營業四部	生管課	行銷部	高雄所
for i in range(1,5):
    print("延遲{}期".format(i))
    lag_period = i
    taipei = np.zeros(7257)        # 各產品贈送量 相關係數array
    taichung = np.zeros(7257)   # 各產品促銷量 相關係數array
    yilan = np.zeros(7257) # 各產品含稅總成本 相關係數array
    factory = np.zeros(7257) # 各產品淨利 相關係數array
    sales_1 = np.zeros(7257) # 各產品成本合計 相關係數array
    sales_3 = np.zeros(7257) # 各產品銷貨淨額 相關係數array
    sales_5 = np.zeros(7257) # 各產品實銷貨量(本身) 相關係數array
    sales_6 = np.zeros(7257) # 各產品實銷貨量(本身) 相關係數array
    sales_4 = np.zeros(7257) # 各產品實銷貨量(本身) 相關係數array
    production_manage = np.zeros(7257) # 各產品實銷貨量(本身) 相關係數array
    marketing = np.zeros(7257) # 各產品實銷貨量(本身) 相關係數array
    kaohsiung = np.zeros(7257) # 各產品實銷貨量(本身) 相關係數array
    for idx, p_name in enumerate(eda_data["產品名稱"].unique()):
        a_product = eda_data[eda_data["產品名稱"] == p_name]
        taipei_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["北營所"][:-lag_period])[0][1]
        if np.isnan(taipei_corr):
            taipei_corr = 0
        taipei[idx] = taipei_corr

        taichung_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["台中所"][:-lag_period])[0][1]
        if np.isnan(taichung_corr):
            taichung_corr = 0
        taichung[idx] = taichung_corr
        
        yilan_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["宜蘭所"][:-lag_period])[0][1]
        if np.isnan(yilan_corr):
            yilan_corr = 0
        yilan[idx] = yilan_corr
        
        factoryt_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["廠務部"][:-lag_period])[0][1]
        if np.isnan(factoryt_corr):
            factoryt_corr = 0
        factory[idx] = factoryt_corr
        
        sales_1_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["營業一部"][:-lag_period])[0][1]
        if np.isnan(sales_1_corr):
            sales_1_corr = 0
        sales_1[idx] = sales_1_corr
        
        sales_3_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["營業三部"][:-lag_period])[0][1]
        if np.isnan(sales_3_corr):
            sales_3_corr = 0
        sales_3[idx] = sales_3_corr
        
        sales_5_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["營業五部"][:-lag_period])[0][1]
        if np.isnan(sales_5_corr):
            sales_5_corr = 0
        sales_5[idx] = sales_5_corr
        
        sales_6_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["營業六部"][:-lag_period])[0][1]
        if np.isnan(sales_6_corr):
            sales_6_corr = 0
        sales_6[idx] = sales_6_corr
        
        sales_4_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["營業四部"][:-lag_period])[0][1]
        if np.isnan(sales_4_corr):
            sales_4_corr = 0
        sales_4[idx] = sales_4_corr
        
        production_manage_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["生管課"][:-lag_period])[0][1]
        if np.isnan(production_manage_corr):
            production_manage_corr = 0
        production_manage[idx] = production_manage_corr
        
        marketing_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["行銷部"][:-lag_period])[0][1]
        if np.isnan(marketing_corr):
            marketing_corr = 0
        marketing[idx] = marketing_corr
        
        kaohsiung_corr = np.corrcoef(a_product["實銷貨量"][lag_period:], a_product["高雄所"][:-lag_period])[0][1]
        if np.isnan(kaohsiung_corr):
            kaohsiung_corr = 0
        kaohsiung[idx] = kaohsiung_corr
        
    print("北營所 vs 實銷貨量 相關係數")
    print(np.mean(taipei))
    print()
    print("台中所 vs 實銷貨量 相關係數")
    print(np.mean(taichung))
    print()
    print("宜蘭所 vs 實銷貨量 相關係數")
    print(np.mean(yilan))
    print()
    print("廠務部 vs 實銷貨量 相關係數")
    print(np.mean(factory))
    print()
    print("營業一部 vs 實銷貨量 相關係數")
    print(np.mean(sales_1))
    print()
    print("營業三部 vs 實銷貨量 相關係數")
    print(np.mean(sales_3))
    print()
    print("營業五部 vs 實銷貨量 相關係數")
    print(np.mean(sales_5))
    print()
    print("營業六部 vs 實銷貨量 相關係數")
    print(np.mean(sales_6))
    print()
    print("營業四部 vs 實銷貨量 相關係數")
    print(np.mean(sales_4))
    print()
    print("生管課 vs 實銷貨量 相關係數")
    print(np.mean(production_manage))
    print()
    print("行銷部 vs 實銷貨量 相關係數")
    print(np.mean(marketing))
    print()
    print("高雄所 vs 實銷貨量 相關係數")
    print(np.mean(kaohsiung))
    print()
    print("-------------------------------------------------------------------------------")

### (3) 資料分群 (最終沒有採用，仍然保留該部分)

In [ ]:
# Flatten to 2D for clustering
nproducts, nsamples, nweeks, nx = X_train_cluster.shape # (7257, 45, 3, 47)
X_train_cluster = X_train_cluster.reshape((nproducts * nsamples, nweeks * nx))
X_test_cluster = X_test_cluster.reshape((nproducts * 1, nweeks * nx))

In [ ]:
# Clustering training data into 2 clusters
#KMeans分成兩類
clf = KMeans(n_clusters=2)
#開始訓練！
clf.fit(X_train_cluster)
#這樣就可以取得預測結果了！
clf.labels_    #共 44 * 7257筆

In [ ]:
print(clf.labels_[:44])
print(clf.labels_[176:220])
print(clf.labels_[-44:])
print(clf.labels_[220:220+44])
print("看0 1 2各自有幾個")
print("0 有 %d 個" % list(clf.labels_).count(0))
print("1 有 %d 個" % list(clf.labels_).count(1))
print("2 有 %d 個" % list(clf.labels_).count(2))
print(len(clf.labels_))
print(len(X_train_cluster))

In [ ]:
# Clustering testing data
test_clf_labels = clf.predict(X_test_cluster)
print(test_clf_labels[:44])
print(test_clf_labels[44:88])
print(test_clf_labels[88:132])
print(test_clf_labels)
print("看0 1 2各自有幾個")
print("0 有 %d 個" % list(test_clf_labels).count(0))
print("1 有 %d 個" % list(test_clf_labels).count(1))
print("2 有 %d 個" % list(test_clf_labels).count(2))
print(len(test_clf_labels))
print(len(X_test_cluster))

In [ ]:
# Labeling
# training data
n_clf = 0 # n-th cluster
n_clf_train_X = []
n_clf_train_Y = []
for i, product in enumerate(X_train_list):
    for j, stack in enumerate(product):
        if clf.labels_[i * len(X_train_list[0]) + j] == n_clf: # len(X_train_list[0]) == 44
            n_clf_train_X.append(stack)
            n_clf_train_Y.append(Y_train_list[i][j])
print(np.array(n_clf_train_X).shape) # (293157, 3, 59)
print(np.array(n_clf_train_Y).shape) # (293157, 1)

In [ ]:
# check labeling (training data)
print(len(n_clf_train_X))

# sum of n-th labels
counter = 0
for l in clf.labels_:
    if l == n_clf:
        counter += 1
print(counter)
print(len(clf.labels_))

In [ ]:
# Labeling
# testing data
n_clf = 0 # n-th cluster
n_clf_test_X = []
n_clf_test_Y = []
for i, product in enumerate(X_test_list):
    if test_clf_labels[i] == n_clf:
        n_clf_test_X.append(product[0])
        n_clf_test_Y.append(target[i]) # Y_test_list[i][0]
print(np.array(n_clf_test_X).shape) # (6816, 3, 59)
print(np.array(n_clf_test_Y).shape) # (6816, 1)

In [ ]:
# check labeling (testing data)
print(len(n_clf_test_X))

# sum of n-th labels
counter = 0
for l in test_clf_labels:
    if l == n_clf:
        counter += 1
print(counter)
print(len(test_clf_labels))

In [ ]:
# reshape data > model data
n_clf_train_X = np.array(n_clf_train_X)
n_clf_train_Y = np.array(n_clf_train_Y)
n_clf_test_X = np.array(n_clf_test_X)
n_clf_test_Y = np.array(n_clf_test_Y)

print(n_clf_train_X.shape, n_clf_train_Y.shape)
print(n_clf_test_X.shape, n_clf_test_Y.shape)

In [ ]:
import sys
for var, obj in locals().items():
    if sys.getsizeof(obj) > 1000000:
        print(var, sys.getsizeof(obj))

In [ ]:
del data, train, X_train_list, Y_train_list, X_test_list
print("done")

## 爬蟲

In [1]:
import requests
from lxml import html
from bs4 import BeautifulSoup as bs

In [2]:
# 抓一般網頁的爬蟲，並未採用
import urllib.request as req
url = "http://statdb.dgbas.gov.tw/pxweb/Dialog/Saveshow.asp"
# 建立一個 request物件，附加 request header的資訊
request = req.Request(url, headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"
    
})
with req.urlopen(request) as response:
    price_data = response.read().decode("utf-8")

print(price_data)

#解析原始碼

# 查詢得到的網頁並非真實網頁，僅是資料庫產生的頁面，無法用爬蟲讀取QQ
# 直接下載到本機端則無法在伺服器上讀取QQ

In [3]:
# 結合外部變數
# 抓物價指數
LOGIN_URL = "https://statdb.mol.gov.tw/statis/jspProxy.aspx"
payload = {
'sys':220,
'ym':10301,
'ymt':10612,
'kind':21,
'type':1,
'funid':'q01014',
'cycle':41,
'outmode':0,
'compmode':0,
'outkind':11,
'fld0':1,
'rdm':'pmh7gfla',
}
r = requests.get(LOGIN_URL,params = payload)
soup = bs(r.text, 'html.parser')

In [4]:
month_list = soup.find_all('th', class_ = 'stycode')
price_index_list = soup.find_all('td', class_ = 'stymon stydata')
month = []
price = []
for m in month_list:
    if " " in m.string:
        month.append(m.string)
for p in price_index_list:
    price.append(p.string)
print(month)
print(price)

['103年 1月', '103年 2月', '103年 3月', '103年 4月', '103年 5月', '103年 6月', '103年 7月', '103年 8月', '103年 9月', '103年 10月', '103年 11月', '103年 12月', '104年 1月', '104年 2月', '104年 3月', '104年 4月', '104年 5月', '104年 6月', '104年 7月', '104年 8月', '104年 9月', '104年 10月', '104年 11月', '104年 12月', '105年 1月', '105年 2月', '105年 3月', '105年 4月', '105年 5月', '105年 6月', '105年 7月', '105年 8月', '105年 9月', '105年 10月', '105年 11月', '105年 12月', '106年 1月', '106年 2月', '106年 3月', '106年 4月', '106年 5月', '106年 6月', '106年 7月', '106年 8月', '106年 9月', '106年 10月', '106年 11月', '106年 12月']
[' 98.25 ', ' 97.87 ', ' 98.05 ', ' 98.71 ', ' 98.86 ', ' 99.37 ', ' 99.34 ', ' 99.77 ', ' 99.72 ', ' 99.79 ', ' 98.88 ', ' 98.54 ', ' 97.33 ', ' 97.67 ', ' 97.45 ', ' 97.90 ', ' 98.14 ', ' 98.81 ', ' 98.71 ', ' 99.33 ', ' 100.01 ', ' 100.10 ', ' 99.41 ', ' 98.67 ', ' 98.12 ', ' 100.02 ', ' 99.41 ', ' 99.73 ', ' 99.35 ', ' 99.70 ', ' 99.93 ', ' 99.89 ', ' 100.35 ', ' 101.80 ', ' 101.37 ', ' 100.34 ', ' 100.32 ', ' 99.96 ', ' 99.59 ', ' 99.83 ', ' 99.94 ',

In [12]:
# 取得食品、飲料及菸草製品批發業成功版本，法2這個則沒有使用次數限制
# 取102~104

LOGIN_URL = "http://dmz21.moea.gov.tw/GA/template/changeData"
payload = {
'url': 'b02',
'selectMultipleMenuItemGoods': 'all',
'selectMultipleMenuItemArea': 'all',
'module4Limit': 15,
'end': 10401,
'limit': 10,
'type': 'm1',
'module5dimensionCode1': 'X10D454'
}
r = requests.post(LOGIN_URL,params = payload)
soup = BeautifulSoup(r.text, 'html.parser')

KeyboardInterrupt: 

In [13]:
# 取105~107
LOGIN_URL = "http://dmz21.moea.gov.tw/GA/template/changeData"
payload = {
'url': 'b02',
'selectMultipleMenuItemGoods': 'all',
'selectMultipleMenuItemArea': 'all',
'module4Limit': 15,
'end': 10701,
'limit': 10,
'type': 'm1',
'module5dimensionCode1': 'X10D454'
}
print()
r = requests.post(LOGIN_URL,params = payload)
soup = BeautifulSoup(r.text, 'html.parser')

KeyboardInterrupt: 

In [ ]:
monthList = []
taiwanWholeSaleOfFoodIndexList = []
for i in soup.select('x')[:36]:
    i = str(i).lstrip("<x>")
    i = i.rstrip("</x>")
    monthList.append(i)
for i in soup.select('y')[:36]:
    i = str(i).lstrip("<y>")
    i = i.rstrip("</y>")
    taiwanWholeSaleOfFoodIndexList.append(i)
print(monthList)
print(taiwanWholeSaleOfFoodIndexList)

## 外部變數欄位(歡迎直接取用)

### 1. 物價指數

In [ ]:
# 物價指數(2013/9~2013/12)： 99.02, 98.75, 98.04, 97.95

# 2014/01~2017/12
price_index = [98.25, 97.87, 98.05, 98.71, 98.86, 99.37, 99.34, 99.77, 99.72, 99.79, 98.88, 98.54, 97.33, 97.67, 97.45, 97.9, 98.14, 98.81, 98.71, 99.33, 100.01, 100.1, 99.41, 98.67, 98.12, 100.02, 99.41, 99.73, 99.35, 99.7, 99.93, 99.89, 100.35, 101.8, 101.37, 100.34, 100.32, 99.96, 99.59, 99.83, 99.94, 100.7, 100.7, 100.85, 100.84, 101.46, 101.71, 101.56]
print(len(price_index))

# 2013/12~2017/11
price_index_lag1 = [97.95, 98.25, 97.87, 98.05, 98.71, 98.86, 99.37, 99.34, 99.77, 99.72, 99.79, 98.88, 98.54, 97.33, 97.67, 97.45, 97.9, 98.14, 98.81, 98.71, 99.33, 100.01, 100.1, 99.41, 98.67, 98.12, 100.02, 99.41, 99.73, 99.35, 99.7, 99.93, 99.89, 100.35, 101.8, 101.37, 100.34, 100.32, 99.96, 99.59, 99.83, 99.94, 100.7, 100.7, 100.85, 100.84, 101.46, 101.71]
print(len(price_index_lag1))

### 2. 食品、飲料及菸草製品零售業營業額

In [ ]:
# 食品、飲料及菸草製品零售業營業額(2013/9~2013/12)： 862, 880, 846, 920

# 2014/01~2017/12
retail_revenue = [955, 754, 852, 860, 874, 874, 959, 925, 889, 876, 865, 980, 957, 843, 844, 817, 846, 877, 925, 941, 899, 868, 869, 962, 953, 819, 854, 859, 892, 937, 979, 1033, 919, 921, 923, 1004, 1012, 766, 914, 860, 990, 991, 939, 1012, 985, 901, 955, 997]
print(len(food_drink_tobacco_product_retail_revenue))

# 2013/12~2017/11
retail_revenue_lag1 = [920, 955, 754, 852, 860, 874, 874, 959, 925, 889, 876, 865, 980, 957, 843, 844, 817, 846, 877, 925, 941, 899, 868, 869, 962, 953, 819, 854, 859, 892, 937, 979, 1033, 919, 921, 923, 1004, 1012, 766, 914, 860, 990, 991, 939, 1012, 985, 901, 955]
print(len(food_drink_tobacco_product_retail_revenue_lag1))

### 3. 該月份是否為特定農曆節日的月份以及特定農曆節日的預熱期(前一個月) (2014/01~2017/12)

In [ ]:
# 春節 農曆 (暫定除夕、初一~初三，不然每年都不一樣?) 農曆12/30 1/1~1/3
is_this_or_next_month_Chinese_New_Year = [1,1,0,0,0,0,0,0,0,0,0,0, 1,1,0,0,0,0,0,0,0,0,0,0, 1,1,0,0,0,0,0,0,0,0,0,1, 1,0,0,0,0,0,0,0,0,0,0,0]
# 元宵節 農曆 農曆1月15日
is_this_or_next_month_Lantern_Festival = [1,1,0,0,0,0,0,0,0,0,0,0, 0,1,1,0,0,0,0,0,0,0,0,0, 1,1,0,0,0,0,0,0,0,0,0,0, 1,1,0,0,0,0,0,0,0,0,0,0]
# 清明節 農曆 (清明不是農曆節日喔，但我還是照做，但不推薦用) 國曆4月3日 or 4日
is_this_or_next_month_Tomb_Sweeping_Day = [0,0,1,1,0,0,0,0,0,0,0,0, 0,0,1,1,0,0,0,0,0,0,0,0, 0,0,1,1,0,0,0,0,0,0,0,0, 0,0,1,1,0,0,0,0,0,0,0,0]
# 端午節 農曆5月5日
is_this_or_next_month_Dragon_Boat_Festival = [0,0,0,0,1,1,0,0,0,0,0,0, 0,0,0,0,1,1,0,0,0,0,0,0, 0,0,0,0,1,1,0,0,0,0,0,0, 0,0,0,1,1,0,0,0,0,0,0,0]
# 中元節 農曆7月15日
is_this_or_next_month_Hungry_Ghost_Festival = [0,0,0,0,0,0,1,1,0,0,0,0, 0,0,0,0,0,0,1,1,0,0,0,0, 0,0,0,0,0,0,1,1,0,0,0,0, 0,0,0,0,0,0,0,1,1,0,0,0]
# 中秋節 農曆8月15日
is_this_or_next_month_Moon_Festival = [0,0,0,0,0,0,0,1,1,0,0,0, 0,0,0,0,0,0,0,1,1,0,0,0, 0,0,0,0,0,0,0,1,1,0,0,0, 0,0,0,0,0,0,0,0,1,1,0,0]

In [ ]:
# 其他不太會影響食品銷量的農曆節日
# 七夕 農曆7月7日
# 重陽節 農曆9月9日

## 清除指定gpu中的模型

In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [ ]:
import gc
from keras import backend as K
K.clear_session()
gc.collect()
#del model